<a href="https://colab.research.google.com/github/stakunlena/ich_final_project/blob/main/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предобработка данных

## Загрузка библиотек и исходных данных

In [345]:
# Импортируем основные библиотеки
import pandas as pd
import numpy as np
from datetime import datetime

# Загружаем исходные данные
import os
from google.colab import drive # Импортируем библиотеку для работы с Google Drive

# Подключаем Google Drive
drive.mount('/content/drive')

# Путь к папке с данными
base_path = '/content/drive/MyDrive/P. Project 07.11/Data/'

# Загружаем файлы в датафреймы
df_calls = pd.read_excel(base_path + 'Calls (Done).xlsx')
df_contacts = pd.read_excel(base_path + 'Contacts (Done).xlsx')
df_deals = pd.read_excel(base_path + 'Deals (Done).xlsx', dtype={"Id": str, "Contact Name": str})
df_spend = pd.read_excel(base_path + 'Spend (Done).xlsx')

# Определим словарь: имя переменной — объект DataFrame
# которым будем пользоваться для групповых операций с датафреймами
dfs = {
    'df_contacts': df_contacts,
    'df_calls': df_calls,
    'df_deals': df_deals,
    'df_spend': df_spend
}

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Кастомные функции

In [346]:
# === Обработка названий колонок ===
import re

def clean_columns_names(df: pd.DataFrame) -> pd.DataFrame:
    """
    Преобразует названия столбцов DataFrame в формат snake_case.

    В текущих датасетах выполняются следующие преобразования:
    1. Удаление лишних пробелов по краям.
    2. Замена пробелов и круглых скобок на символ подчёркивания.
    3. Приведение всех символов к нижнему регистру.

    Параметры
    ----------
    df : pandas.DataFrame
        Исходный DataFrame с оригинальными названиями столбцов.

    Возвращает
    ----------
    pandas.DataFrame
        Тот же объект DataFrame с обновлёнными названиями столбцов.
    """
    df.columns = (
        df.columns
        .map(lambda x: str(x).strip()) # обрезаем пробелы и защищаем от NaN
        .map(lambda x: re.sub(r'[\s()]+', '_', x)) # пробелы и скобки на "_"
        .str.lower() # всё в нижний регистр
    )
    return df

# === Общие функции очистки и предобработки данных

def convert_column_to_string(df: pd.DataFrame, column: str) -> pd.DataFrame:
    """
    Безопасно преобразует указанный столбец в строковый тип pandas.StringDtype().
    Сохраняет пропуски как pd.NA, удаляет .0, пробелы и строковые маркеры NaN.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм.
    column : str
        Название столбца, который требуется преобразовать.

    Возвращает
    ----------
    pd.DataFrame
        Датафрейм с обновлённым столбцом.
    """
    if column not in df.columns:
        print(f"Столбец '{column}' отсутствует в датафрейме, пропуск обработки.")
        return df

    df[column] = (
        df[column]
        .astype("string")
        .str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .replace({"nan": pd.NA, "NaN": pd.NA, "<NA>": pd.NA})
    )
    return df

def analyze_missing_ratio(df: pd.DataFrame, column: str, threshold: float = 0.01) -> None:
    """
    Анализирует долю пропусков в заданной колонке датафрейма и выводит
    рекомендации по дальнейшей обработке строк с пропущенными значениями.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм для анализа.
    column : str
        Название столбца, в котором нужно проверить пропуски.
    threshold : float, optional
        Пороговое значение доли пропусков (по умолчанию 0.1 = 10%).
        Если доля пропусков меньше порога — рекомендуется удалить строки.
        Если доля выше порога — рекомендуется сохранить строки.

    Возвращает
    ----------
    None
        Функция только выводит информацию и не изменяет датафрейм.

    Пример
    -------
    >>> analyze_missing_ratio(df_calls, "contactid", threshold=0.1)
    Количество строк с пропусками в колонке 'contactid': 3933
    Доля пропусков от общего числа строк: 4.10%
    Доля пропусков меньше порогового значения (10.00%). Строки можно удалить.
    """
    # Проверка, что колонка существует
    if column not in df.columns:
        print(f"Ошибка: в датафрейме нет колонки '{column}'.")
        return

    # Подсчёт пропусков
    missing_rows = df[df[column].isna()]
    total_rows = len(df)
    missing_count = len(missing_rows)
    missing_share = missing_count / total_rows if total_rows > 0 else 0

    # Вывод статистики
    print(f"Количество строк с пропусками в колонке '{column}': {missing_count}")
    print(f"Доля пропусков от общего числа строк: {missing_share:.2%}")

    # Формирование рекомендации
    if missing_share < threshold:
        print(f"Доля пропусков меньше порогового значения ({threshold:.2%}). Строки можно удалить.")
    else:
        print(f"Доля пропусков превышает пороговое значение ({threshold:.2%}). Строки лучше сохранить.")

def drop_full_duplicates(df: pd.DataFrame, df_name: str = "DataFrame") -> pd.DataFrame:
    """
    Проверяет и удаляет полные дубликаты строк в переданном датафрейме.

    Функция подсчитывает количество полностью совпадающих строк,
    выводит информацию о результатах проверки и возвращает очищенный датафрейм.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм, в котором нужно найти и удалить дубликаты.
    df_name : str, optional
        Название датафрейма (для удобного вывода в логах), по умолчанию "DataFrame".

    Возвращает
    ----------
    pd.DataFrame
        Копия исходного датафрейма без полных дублей, с обновлёнными индексами.
    """
    duplicates_count = df.duplicated().sum()
    print(f"[{df_name}] Найдено полных дублей: {duplicates_count}")

    if duplicates_count > 0:
        df = df.drop_duplicates().reset_index(drop=True)
        print(f"[{df_name}] Полные дубли удалены. Размер после очистки: {df.shape}")
    else:
        print(f"[{df_name}] Полных дублей не обнаружено. Размер датафрейма: {df.shape}")

    return df

def convert_datetime_columns(df: pd.DataFrame, date_columns: list[str], df_name: str = "DataFrame") -> pd.DataFrame:
    """
    Преобразует указанные столбцы датафрейма в формат datetime64[ns].

    Функция безопасно приводит текстовые значения к типу datetime,
    используя формат '%d.%m.%Y %H:%M' (CRM-формат),
    и сообщает количество некорректных значений.

    Параметры
    ----------
    df : pd.DataFrame
        Исходный датафрейм для преобразования.
    date_columns : list[str]
        Список названий столбцов, которые нужно преобразовать.
    df_name : str, optional
        Название датафрейма (для логирования), по умолчанию "DataFrame".

    Возвращает
    ----------
    pd.DataFrame
        Копия датафрейма с обновлёнными типами указанных столбцов.
    """
    for col in date_columns:
        if col in df.columns:
            #df[col] = pd.to_datetime(df[col], format="%d.%m.%Y %H:%M", errors="coerce")
            df[col] = pd.to_datetime(
              df[col],
              errors="coerce",
              infer_datetime_format=True,
              dayfirst=True,
              utc=False
            )
            invalid = df[col].isna().sum()
            print(f"[{df_name}] Столбец '{col}' преобразован в datetime64[ns]. Некорректных значений: {invalid}")
        else:
            print(f"[{df_name}] Столбец '{col}' не найден, пропускаем.")
    return df

def remove_boolean_artifacts(df, col_name):
    """
    Удаляет строки, где в текстовом столбце встречаются значения типа bool (True/False).
    Используется для очистки артефактов в owner_name, source, campaign и т.п.
    """
    before = len(df)
    df = df[df[col_name] != False].reset_index(drop=True)
    after = len(df)
    removed = before - after
    if removed > 0:
        print(f"Удалено строк с артефактом False в '{col_name}': {removed}")
    else:
        print(f"Артефактов False в '{col_name}' не обнаружено.")
    df[col_name] = df[col_name].astype("string")
    return df


def normalize_categorical_columns(df, columns):
    """
    Приводит строковые категориальные столбцы к единому виду:
    - удаляет лишние пробелы
    - приводит к нижнему регистру
    - заменяет множественные пробелы одним
    """
    for col in columns:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype("string")
                .str.strip()
                .str.lower()
                .str.replace(r"\s+", " ", regex=True)
            )
            print(f"Столбец '{col}' нормализован.")
        else:
            print(f"Столбец '{col}' не найден, пропущен.")
    return df


def fill_missing_numeric(df, col, strategy="zero"):
    """
    Заполняет пропуски в числовом столбце по выбранной стратегии:
    - 'zero' заменить на 0
    - 'mean' заменить на среднее значение
    - 'median' заменить на медиану
    """
    if col not in df.columns:
        print(f"Столбец '{col}' отсутствует.")
        return df

    missing_before = df[col].isna().sum()

    if strategy == "zero":
        df[col] = df[col].fillna(0)
    elif strategy == "mean":
        df[col] = df[col].fillna(df[col].mean())
    elif strategy == "median":
        df[col] = df[col].fillna(df[col].median())
    else:
        print(f"Неизвестная стратегия: {strategy}")
        return df

    missing_after = df[col].isna().sum()
    filled = missing_before - missing_after
    print(f"'{col}': заполнено {filled} пропусков (стратегия='{strategy}')")
    return df


def drop_empty_columns(df, cols):
    """
    Удаляет указанные неинформативные столбцы, если они присутствуют.
    """
    df = df.drop(columns=cols, errors="ignore")
    print(f"Удалены пустые/технические столбцы (если были): {cols}")
    return df

# == Функции экспорта датафреймов

def export_dataframe_to_csv(df: pd.DataFrame, df_name: str, folder_path: str = "/content/drive/MyDrive/P. Project 07.11/csv/") -> str:
    """
    Экспортирует очищенный датафрейм в CSV-файл с меткой времени.

    Функция сохраняет переданный датафрейм в указанный каталог.
    Если папка не существует, она создаётся автоматически.
    Имя файла формируется в формате:
        <df_name>_clean_<YYYYMMDD_HHMM>.csv

    Параметры
    ----------
    df : pd.DataFrame
        Датафрейм, который нужно сохранить.
    df_name : str
        Имя датафрейма (используется в названии файла).
    folder_path : str, optional
        Путь к папке для сохранения. По умолчанию: "/content/drive/MyDrive/P. Project 07.11/csv/"

    Возвращает
    ----------
    str
        Полный путь к сохранённому файлу.
    """

    from datetime import datetime
    import os

    # Создаём папку, если её нет
    os.makedirs(folder_path, exist_ok=True)

    # Формируем имя файла с меткой времени
    timestamp = datetime.now().strftime("%Y%m%d_%H%M")
    output_path = os.path.join(folder_path, f"{df_name}_clean_{timestamp}.csv")

    # Сохраняем CSV
    df.to_csv(output_path, index=False, sep=";", encoding="utf-8-sig")

    print(f"Файл '{df_name}' успешно сохранён по пути:\n{output_path}")
    return output_path

## Ревью исходных данных. Определение количества пропусков

In [347]:
# Вывод информации по каждому датафрейму в цикле по элементам словаря dfs
for name, df in dfs.items():
    print(f"\n{name}")
    #print(f"  Размер: {df.shape[0]} строк × {df.shape[1]} столбцов")
    #print(f"  Колонки: {df.columns.tolist()}")
    print(f"\n")
    print(f"  Информация о датасете:")
    print(df.info())
    print(f"\n")
    print(f"  Первые 5 строк датасета:")
    display(df.head())
    print(f"\n")
    print(f"  Количество пропусков в данных:")
    print(df.isna().sum())
    print(f"\n================================================")


df_contacts


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Id                  18548 non-null  int64 
 1   Contact Owner Name  18548 non-null  object
 2   Created Time        18548 non-null  object
 3   Modified Time       18548 non-null  object
dtypes: int64(1), object(3)
memory usage: 579.8+ KB
None


  Первые 5 строк датасета:


,Id,Contact Owner Name,Created Time,Modified Time
0,5805028000000645014,Rachel White,27.06.2023 11:28,22.12.2023 13:34
1,5805028000000872003,Charlie Davis,03.07.2023 11:31,21.05.2024 10:23
2,5805028000000889001,Bob Brown,02.07.2023 22:37,21.12.2023 13:17
3,5805028000000907006,Bob Brown,03.07.2023 05:44,29.12.2023 15:20
4,5805028000000939010,Nina Scott,04.07.2023 10:11,16.04.2024 16:14




  Количество пропусков в данных:
Id                    0
Contact Owner Name    0
Created Time          0
Modified Time         0
dtype: int64


df_calls


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          95874 non-null  int64  
 1   Call Start Time             95874 non-null  object 
 2   Call Owner Name             95874 non-null  object 
 3   CONTACTID                   91941 non-null  float64
 4   Call Type                   95874 non-null  object 
 5   Call Duration (in seconds)  95791 non-null  float64
 6   Call Status                 95874 non-null  object 
 7   Dialled Number              0 non-null      float64
 8   Outgoing Call Status        86875 non-null  object 
 9   Scheduled in CRM            86875 non-null  float64
 10  Tag                 

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Dialled Number,Outgoing Call Status,Scheduled in CRM,Tag
0,5805028000000805001,30.06.2023 08:43,John Doe,NaN,Inbound,171.0,Received,NaN,NaN,NaN,NaN
1,5805028000000768006,30.06.2023 08:46,John Doe,NaN,Outbound,28.0,Attended Dialled,NaN,Completed,0.0,NaN
2,5805028000000764027,30.06.2023 08:59,John Doe,NaN,Outbound,24.0,Attended Dialled,NaN,Completed,0.0,NaN
3,5805028000000787003,30.06.2023 09:20,John Doe,5.805028e+18,Outbound,6.0,Attended Dialled,NaN,Completed,0.0,NaN
4,5805028000000768019,30.06.2023 09:30,John Doe,5.805028e+18,Outbound,11.0,Attended Dialled,NaN,Completed,0.0,NaN




  Количество пропусков в данных:
Id                                0
Call Start Time                   0
Call Owner Name                   0
CONTACTID                      3933
Call Type                         0
Call Duration (in seconds)       83
Call Status                       0
Dialled Number                95874
Outgoing Call Status           8999
Scheduled in CRM               8999
Tag                           95874
dtype: int64


df_deals


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21595 entries, 0 to 21594
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Id                   21593 non-null  object 
 1   Deal Owner Name      21564 non-null  object 
 2   Closing Date         14645 non-null  object 
 3   Quality              19340 non-null  object 
 4   Stage                21593 non-null  object 
 5   Lost Reason          16124 non-null  object 
 6   Page

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,...,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch
0,5805028000056864695,Ben Hall,NaN,NaN,New Lead,NaN,/eng/test,03.07.23women,NaN,v16,...,NaN,NaN,21.06.2024 15:30,NaN,NaN,NaN,NaN,5805028000056849495,NaN,NaN
1,5805028000056859489,Ulysses Adams,NaN,NaN,New Lead,NaN,/at-eng,NaN,NaN,NaN,...,Web Developer,Morning,21.06.2024 15:23,6.0,NaN,0,2000,5805028000056834471,NaN,NaN
2,5805028000056832357,Ulysses Adams,21.06.2024,D - Non Target,Lost,Non target,/at-eng,engwien_AT,00:26:43,b1-at,...,NaN,NaN,21.06.2024 14:45,NaN,NaN,NaN,NaN,5805028000056854421,NaN,NaN
3,5805028000056824246,Eva Kent,21.06.2024,E - Non Qualified,Lost,Invalid number,/eng,04.07.23recentlymoved_DE,01:00:04,bloggersvideo14com,...,NaN,NaN,21.06.2024 13:32,NaN,NaN,NaN,NaN,5805028000056889351,NaN,NaN
4,5805028000056873292,Ben Hall,21.06.2024,D - Non Target,Lost,Non target,/eng,discovery_DE,00:53:12,website,...,NaN,NaN,21.06.2024 13:21,NaN,NaN,NaN,NaN,5805028000056876176,NaN,NaN




  Количество пропусков в данных:
Id                         2
Deal Owner Name           31
Closing Date            6950
Quality                 2255
Stage                      2
Lost Reason             5471
Page                       2
Campaign                5528
SLA                     6062
Content                 7448
Term                    9141
Source                     2
Payment Type           21099
Product                18003
Education Type         18295
Created Time               2
Course duration        18008
Months of study        20755
Initial Amount Paid    17430
Offer Total Amount     17410
Contact Name              63
City                   19084
Level of Deutsch       20344
dtype: int64


df_spend


  Информация о датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20779 entries, 0 to 20778
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         20779 non-null  datetim

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
0,2023-07-03,Google Ads,gen_analyst_DE,6,0.00,0,NaN,NaN
1,2023-07-03,Google Ads,performancemax_eng_DE,4,0.01,1,NaN,NaN
2,2023-07-03,Facebook Ads,NaN,0,0.00,0,NaN,NaN
3,2023-07-03,Google Ads,NaN,0,0.00,0,NaN,NaN
4,2023-07-03,CRM,NaN,0,0.00,0,NaN,NaN




  Количество пропусков в данных:
Date              0
Source            0
Campaign       5994
Impressions       0
Spend             0
Clicks            0
AdGroup        6828
Ad             6828
dtype: int64



## Приведение названия колонок всех датасетов к snake_case

In [348]:
# Изменение названий колонок в датафреймах с помощью функции clean_columns_names
for name, df in dfs.items():
    clean_columns_names(df)
    print(f"\n{name}")
    print(list(df.columns))


df_contacts
['id', 'contact_owner_name', 'created_time', 'modified_time']

df_calls
['id', 'call_start_time', 'call_owner_name', 'contactid', 'call_type', 'call_duration_in_seconds_', 'call_status', 'dialled_number', 'outgoing_call_status', 'scheduled_in_crm', 'tag']

df_deals
['id', 'deal_owner_name', 'closing_date', 'quality', 'stage', 'lost_reason', 'page', 'campaign', 'sla', 'content', 'term', 'source', 'payment_type', 'product', 'education_type', 'created_time', 'course_duration', 'months_of_study', 'initial_amount_paid', 'offer_total_amount', 'contact_name', 'city', 'level_of_deutsch']

df_spend
['date', 'source', 'campaign', 'impressions', 'spend', 'clicks', 'adgroup', 'ad']


## Обработка PK и FK таблиц

Проанализируем первичные и вторичные ключи исходных таблиц:


| Датафрейм | Ключ (PK/FK) | Тип PK/FK | Количество<br>пропусков | Планируемые действия |
|-|-|-|-|-|
| df_calls | id (PK)| int64 | 0 | Преобразовать в string |
| df_calls | contactid (FK)| float64 | 3933 | Преобразовать в string с сохранением пропусков |
| df_contacts | id (PK) | int64 | 0 | Преобразовать в string |
| df_deals | id (PK) | float64 | 2 | Удалить строки с пропусками, затем преобразовать в string |
| df_deals | contact_name (FK) | float64 | 63 | Преобразовать в string с сохранением пропусков |
| df_spend | — | — | — | Пока ничего не делаем |

Выводы:
* для исключает риск округления и потери разрядов и обеспечения корректного объединения таблиц (merge, join) нужно преобразовать поля PK/FK в тип string;
* в датафрейме df_calls обнаружены 3933 строк с пропусками во вторичном ключе contactid — их слишком много, чтобы удалить данные строки и надо преобразовать поле в формат string с сохранением пропусков;
* в датафрейме df_deals обнаружены 2 строки с пропусками в первичном ключе, скорее всего это «мусор» в данных, перед преобразованием в тип string их надо проанализировать и удалить, если они не содержат значимой информации;
* в датафрейме df_deals обнаружены 63 строки с пропусками во вторичном ключе, пока не трогаем их;
* в датафрейме df_spend специальных полей с идентификаторами нет, возможно понадобится впоследствии добавить в нее первичный ключ.

### Анализ и обработка пропусков в contactid (FK) датафрейма df_calls

In [349]:
analyze_missing_ratio(df_calls, "contactid")

Количество строк с пропусками в колонке 'contactid': 3933
Доля пропусков от общего числа строк: 4.10%
Доля пропусков превышает пороговое значение (1.00%). Строки лучше сохранить.


Выводы:
* доля строк с пропущенным contactid слишком велика, чтобы удалять их исходных данных;
* следует преобразовать это поле к типу strig с сохранением пропусков.

### Анализ и обработка пропусков в PK датафрейма df_deals

In [350]:
# Отфильтровать строки с пропусками в Id
missing_deals_id_rows = df_deals[df_deals['id'].isna()]

# Посмотреть количество, их долю и сами строки
analyze_missing_ratio(df_calls, "id")
#print(f"Количество строк в df_deals с пропущенным Id: {len(missing_deals_id_rows)}\n")
#print(f"Доля строк с пропущенным Id: {len(missing_deals_id_rows) / len(df_deals):.2%}\n")
print(f"Содержимое строк с пропущенным Id:\n\n{missing_deals_id_rows}")

Количество строк с пропусками в колонке 'id': 0
Доля пропусков от общего числа строк: 0.00%
Доля пропусков меньше порогового значения (1.00%). Строки можно удалить.
Содержимое строк с пропущенным Id:

        id deal_owner_name closing_date quality stage lost_reason page  \
21593  NaN             NaN          NaN     NaN   NaN         NaN  NaN   
21594  NaN             NaN          NaN     NaN   NaN         NaN  NaN   

      campaign  sla content  ... product education_type created_time  \
21593      NaN  NaN     NaN  ...     NaN            NaN          NaN   
21594      NaN  NaN     NaN  ...     NaN          #REF!          NaN   

      course_duration months_of_study initial_amount_paid  offer_total_amount  \
21593             NaN             NaN                 NaN                 NaN   
21594             NaN             NaN                 NaN                 NaN   

       contact_name city level_of_deutsch  
21593           NaN  NaN              NaN  
21594           NaN  NaN   

Выводы по результатам анализа пропусков в PK в df_deals:
* в таблице df_deals есть 2 строки, в которых одновременно пропущены все ключевые поля: Id, Stage, Page, Source, Created Time (и фактически не содержат ни одного полезного значени);
* индексы этих строк: 21593 и 21594;
* это типичные артефакты Excel-выгрузки — «пустые строки внизу файла»;
* эти строки следует удалить как технический мусор;
* строка 21594 содержит в колонке Education Type содержится значение #REF! — это помешает удалить её методом dropna и перед этим надо заменить текстовые «артефакты Excel» (#REF!, #N/A, #VALUE!, nan, и т.д.) на настоящие пропуски (pd.NA)

In [351]:
# Заменяем текстовые артефакты Excel на NaN
df_deals = df_deals.replace(['#REF!', '#N/A', '#VALUE!', 'NaN', 'nan', ''], np.nan)

# Удаляем строки с пропусками в PK из df_deals
df_deals = df_deals.dropna(how='all').reset_index(drop=True)

# Проверяем результат
print(df_deals.isna().sum())

id                         0
deal_owner_name           29
closing_date            6948
quality                 2253
stage                      0
lost_reason             5469
page                       0
campaign                5526
sla                     6060
content                 7446
term                    9139
source                     0
payment_type           21097
product                18001
education_type         18294
created_time               0
course_duration        18006
months_of_study        20753
initial_amount_paid    17428
offer_total_amount     17408
contact_name              61
city                   19082
level_of_deutsch       20342
dtype: int64


### Преобразование первичных ключей таблиц к типу string

In [352]:
'''
Первая версия обработки с использованием кастомной функции, преобразовывавшей
столбцы содержащие id в заголовке. Подход оказался неудачным, так как функция
не учитывает столбец contact_name, а столбец initial_amount_paid приходится
исключать из обработки вручную.

# Преобразуем столбцы-ключи в таблицах к типу string
for name, df in dfs.items():
    df = normalize_id_columns(df)
    id_cols = [c for c in df.columns if "id" in c.lower()]
    print(name.lower())
    print(f"{df[id_cols].dtypes}\n")

# Проверяем, что в преобразованном столбце сохранились строки с пропусками в contactid
analyze_missing_ratio(df_calls, "contactid")

'''


# === Преобразование PK/FK для всех таблиц проекта ===

## --- df_calls ---
# id (PK)
df_calls = convert_column_to_string(df_calls, "id")

# contactid (FK) — сохраняем пропуски
df_calls = convert_column_to_string(df_calls, "contactid")

## --- df_contacts ---
# id (PK)
df_contacts = convert_column_to_string(df_contacts, "id")

## --- df_deals ---
# id (PK) — удалить строки с пропусками, затем преобразовать
df_deals = df_deals.dropna(subset=["id"]).reset_index(drop=True)
df_deals = convert_column_to_string(df_deals, "id")

# contact_name (FK) — удалить строки с пропусками, затем преобразовать
#df_deals = df_deals.dropna(subset=["contact_name"]).reset_index(drop=True)
df_deals = convert_column_to_string(df_deals, "contact_name")

## --- df_spend ---
# — пока ничего не делаем


# === Проверка преобразования типов ===
def check_key_columns_status(df_dict):
    """
    Выводит тип данных и количество пропусков в ключевых полях
    для всех датафреймов проекта.
    """
    print("\n=== Проверка преобразования ключевых столбцов ===")
    for name, df in df_dict.items():
        key_cols = [c for c in df.columns if c in ["id", "contactid", "contact_name"]]
        if not key_cols:
            continue
        print(f"\n {name}:")
        for col in key_cols:
            missing = df[col].isna().sum()
            total = len(df)
            pct = (missing / total * 100) if total else 0
            print(f"{col}: dtype={df[col].dtype}, пропусков={missing} ({pct:.2f}%)")

# Проверяем сделанное преобразование типов в ключевых столбцах
check_key_columns_status(dfs)




=== Проверка преобразования ключевых столбцов ===

 df_contacts:
id: dtype=string, пропусков=0 (0.00%)

 df_calls:
id: dtype=string, пропусков=0 (0.00%)
contactid: dtype=string, пропусков=3933 (4.10%)

 df_deals:
id: dtype=object, пропусков=2 (0.01%)
contact_name: dtype=object, пропусков=63 (0.29%)


## Предобработка df_contacts (таблица Contacts (Done).xlsx)

|Этап предобработки|Необходимые действия|Ожидаемый результат|
|--|--|--|
|1. Проверка и удаление полных дублей|Проверить датафрейм на наличие полностью идентичных строк с помощью кастомной функции drop_full_duplicates(). Удалить найденные дубликаты|Удалены повторяющиеся записи, каждая строка представляет уникальный контакт|
|2. Преобразование формата дат|Преобразовать столбцы created_time и modified_time к типу datetime64[ns] с указанием формата "%d.%m.%Y %H:%M"|Корректные типы данных, позволяющие выполнять сортировку и анализ по времени|
|3. Проверка логики дат|Проверить, что modified_time не раньше created_time, при нарушениях зафиксировать или удалить строки|Данные согласованы по времени, отсутствуют нелогичные даты|
|4. Очистка имён владельцев контактов и проверка на неявные дубли|Удалить лишние пробелы, заменить двойные пробелы одним, привести к единому регистру методом .str.title(). Сравнить уникальные значения contact_owner_name, при необходимости создать словарь нормализации|Единый формат имён владельцев, готовых для дальнейшего сопоставления с другими таблицами. Исправлены возможные опечатки и вариации написания, снижено количество уникальных значений|
|5. Финальная проверка типов данных|Убедиться, что id имеет строковый тип, contact_owner_name — строковый, даты — datetime|Все столбцы имеют корректные типы данных для анализа и объединений|
|6. Выгрузка очищенного датасета в .csv |Экспортировать датафрейм в файл .csv. Если в папке проекта нет подкаталога /csv то создать его|Файл сохранен в подкаталоге /csv в папке проекта|

Для того, чтобы оптимизировать процеудуру предобработки:
* очистку имён владельцев контактов и проверка на неявные дубли сделаем перед основным блоком преобработки;
* проверку логики дат перенесем на этап EDA.


### Проверка имён владельцев контактов на неявные дубли

In [353]:
unique_count = df_contacts["contact_owner_name"].nunique()
print(f"Количество уникальных имён владельцев контактов: {unique_count}")
print(df_contacts["contact_owner_name"].unique())
print("\nОбнаружен 1 артефакт — значение False.")
count_false = (df_contacts["contact_owner_name"] == False).sum()
print(f"Количество строк с артефактом: {count_false}")

Количество уникальных имён владельцев контактов: 28
['Rachel White' 'Charlie Davis' 'Bob Brown' 'Nina Scott' 'Alice Johnson'
 'Ian Miller' 'Jane Smith' 'Julia Nelson' 'George King' 'Quincy Vincent'
 'Diana Evans' 'Kevin Parker' 'Ulysses Adams' 'Victor Barnes'
 'Yara Edwards' 'Paula Underwood' 'Mason Roberts' 'Ben Hall' 'Amy Green'
 'Cara Iverson' 'Oliver Taylor' 'Eva Kent' False 'Zachary Foster'
 'Sam Young' 'Wendy Clark' 'Tina Zhang' 'Derek James']

Обнаружен 1 артефакт — значение False.
Количество строк с артефактом: 1


Выводы:
* очистка и нормализация имен владельцев контактов не требуется;
* неявные дубли отсутствуют;
* обнаружена одна строка с артефактом — значение False вместо имени — можно удалить.



### Предобработка df_contacts и выгрузка в csv

In [354]:
print("=== 1. Проверка и удаление полных дублей ===\n")
df_contacts = drop_full_duplicates(df_contacts, "df_contacts")

print("\n=== 2. Преобразование формата дат ===\n")
df_contacts = convert_datetime_columns(
    df_contacts,
    ["created_time", "modified_time"],
    df_name="df_contacts"
)

print("\n=== 3. Очистка артефактов в именах владельцев контактов ===\n")
df_contacts = df_contacts[df_contacts["contact_owner_name"] != False].reset_index(drop=True)
df_contacts["contact_owner_name"] = df_contacts["contact_owner_name"].astype("string")
print(df_contacts["contact_owner_name"].unique())

print("\n=== 4. Проверка типов данных перед выгрузкой в .csv ===\n")
print("Текущие типы данных столбцов:\n")
print(f"[df_contacts] Размер датафрейма: {df_contacts.shape}\n")
print(df_contacts.dtypes)

print("\n=== 5. Выгрузка очищенного датасета в .csv ===\n")
export_dataframe_to_csv(df_contacts, "df_contacts")

=== 1. Проверка и удаление полных дублей ===

[df_contacts] Найдено полных дублей: 0
[df_contacts] Полных дублей не обнаружено. Размер датафрейма: (18548, 4)

=== 2. Преобразование формата дат ===

[df_contacts] Столбец 'created_time' преобразован в datetime64[ns]. Некорректных значений: 0
[df_contacts] Столбец 'modified_time' преобразован в datetime64[ns]. Некорректных значений: 0

=== 3. Очистка артефактов в именах владельцев контактов ===

<StringArray>
[   'Rachel White',   'Charlie Davis',       'Bob Brown',      'Nina Scott',
   'Alice Johnson',      'Ian Miller',      'Jane Smith',    'Julia Nelson',
     'George King',  'Quincy Vincent',     'Diana Evans',    'Kevin Parker',
   'Ulysses Adams',   'Victor Barnes',    'Yara Edwards', 'Paula Underwood',
   'Mason Roberts',        'Ben Hall',       'Amy Green',    'Cara Iverson',
   'Oliver Taylor',        'Eva Kent',  'Zachary Foster',       'Sam Young',
     'Wendy Clark',      'Tina Zhang',     'Derek James']
Length: 27, dtype: 

/tmp/ipython-input-2139470510.py:167: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(
/tmp/ipython-input-2139470510.py:167: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(


'/content/drive/MyDrive/P. Project 07.11/csv/df_contacts_clean_20251102_1847.csv'

## Предобработка df_calls (таблица Calls (Done).xlsx)

|Этап предобработки|Необходимые действия|Ожидаемый результат|
|--|--|--|
|1. Проверка и удаление полных дублей|Проверить датафрейм на наличие полностью идентичных строк с помощью кастомной функции drop_full_duplicates(). Удалить найденные дубликаты|Удалены повторяющиеся строки, каждая запись соответствует уникальному звонку|
|2. Преобразование формата даты и времени|Преобразовать столбец call_start_time в формат datetime64[ns] с указанием формата "%d.%m.%Y %H:%M". Проверить корректность диапазона дат и отсутствие будущих значений|Корректный тип данных, позволяющий сортировать и анализировать звонки по времени|
|3. Очистка имён владельцев звонков|Удалить лишние пробелы, заменить двойные пробелы одним, привести значения к единому регистру методом .str.title(). Проверить список уникальных имён на наличие опечаток и дублей|Единый формат имён сотрудников, готовый для анализа активности и объединений с другими таблицами|
|4. Восстановление пропусков contactid|Попытаться восстановить пропуски contactid по таблице contacts при совпадении владельца звонка с владельцем контакта|Максимально заполненные идентификаторы контактов без потери точности значений|
|5. Проверка и очистка категориальных значений|Проверить уникальные значения столбцов call_type, call_status, outgoing_call_status на опечатки и различия в регистре. Привести к унифицированному набору категорий|Корректные и стандартизированные категориальные значения, готовые к агрегации|
|6. Обработка длительности звонков|Проверить столбец call_duration_in_seconds_ на наличие пропусков и нулевых значений. Заменить NaN на 0. Проверить согласованность длительности с call_status (attended не может иметь 0 секунд)|Отсутствуют пропуски, длительность соответствует логике состояний звонков|
|7. Проверка несогласованных статусов|Проверить случаи, где одновременно присутствуют inbound и outbound звонки для одного контакта и времени. Добавить флаг is_multi_type_call для таких строк|Зафиксированы и помечены технические дубликаты событий CRM|
|8. Удаление полностью пустых столбцов|Удалить столбцы dialled_number и tag, содержащие 100% пропусков|Исключены неинформативные поля, датафрейм содержит только полезные данные|
|9. Проверка типов данных|Проверить, что id и contactid имеют строковый тип, call_start_time — datetime, количественные поля — числовой тип|Все поля имеют корректные типы для анализа и объединений|
|10. Выгрузка очищенного датасета в .csv|Экспортировать df_calls в файл .csv. Если в папке проекта отсутствует подкаталог /csv, создать его|Файл сохранён в подкаталоге /csv в папке проекта|

Для того, чтобы оптимизировать процеудуру предобработки 7 этап «Проверка несогласованных статусов» перенесем на этап EDA.

### Проверка логики показателя scheduled_in_crm

В большинстве CRM-систем:
* 1 — звонок был запланирован в календаре или задаче CRM (назначен заранее менеджером);
* 0 — звонок не был запланирован, а создан постфактум (например, ручной звонок или входящий).

Тем не менее, чтобы делать корректные выводы о статистике запланированных звонков требуется убедиться, что в нашем случае эта логика работает.

Для этого построим кросстабы по типам звонков и scheduled_in_crm и по статусам звонков и scheduled_in_crm.

In [355]:
print("=== Проверка логики scheduled_in_crm ===\n")

# Пересечение по типам звонков
print("=== Пересечение по типам звонков ===")
print(pd.crosstab(df_calls["call_type"], df_calls["scheduled_in_crm"]))

# Пересечение по статусам
print("\n=== Пересечение по статусам ===")
print(pd.crosstab(df_calls["call_status"], df_calls["scheduled_in_crm"]))


=== Проверка логики scheduled_in_crm ===

=== Пересечение по типам звонков ===
scheduled_in_crm    0.0  1.0
call_type                   
Outbound          86733  142

=== Пересечение по статусам ===
scheduled_in_crm              0.0  1.0
call_status                           
Attended Dialled            70703    0
Cancelled                       0   20
Overdue                         0   60
Scheduled                       0    3
Scheduled Attended              0   14
Scheduled Attended Delay        0   22
Scheduled Unattended            0    6
Scheduled Unattended Delay      0   17
Unattended Dialled          16030    0


Видим, что scheduled_in_crm = 1:
*   встречаются только у исходящих (outbound) звонков;
*   только у звонков со статусами отменено (cancelled), просроченный (overdue) и разнообразные виды запланировано (scheduled).

Вывод: логика значений scheduled_in_crm:
*   0 = «Нет»;
*   1 = «Да».



### Восстанавление пропущенных contactid в таблице звонков по совпадению имён владельцев

In [356]:
def restore_contactid_from_owner(df_calls, df_contacts):
    """
    Восстанавливает пропущенные contactid в таблице звонков по совпадению имён владельцев.
    Использует словарь соответствий contact_owner_name полю id из df_contacts.
    """
    print("Восстановление пропущенных contactid по contact_owner_name...")
    missing_before = df_calls["contactid"].isna().sum()

    owner_to_contactid = (
        df_contacts[["contact_owner_name", "id"]]
        .drop_duplicates(subset=["contact_owner_name"])
        .set_index("contact_owner_name")["id"]
        .to_dict()
    )

    mask_missing = df_calls["contactid"].isna()
    df_calls.loc[mask_missing, "contactid"] = (
        df_calls.loc[mask_missing, "call_owner_name"].map(owner_to_contactid)
    )

    missing_after = df_calls["contactid"].isna().sum()
    filled = missing_before - missing_after

    print(f"Восстановлено contactid: {filled} | Осталось пропусков: {missing_after}")
    return df_calls

### Преобразование бинарного столбца scheduled_in_crm в числовой (0/1)

In [357]:
def process_scheduled_flag(df, col="scheduled_in_crm"):
    """
    Преобразует бинарный столбец scheduled_in_crm в числовой (0/1).
    Заполняет пропуски нулями, использует компактный тип int8.
    """
    if col not in df.columns:
        print(f"Столбец '{col}' отсутствует, пропущен.")
        return df

    df[col] = df[col].fillna(0).astype("int8")
    print(f"Столбец '{col}' преобразован в логический формат (0/1).")
    print(df[col].value_counts(dropna=False))
    return df

### Предобработка df_calls и выгрузка в csv

In [358]:
# === ОСНОВНОЙ ПАЙПЛАЙН ПРЕДОБРАБОТКИ ДАННЫХ DF_CALLS ===

print("=== 1. Удаление полных дублей ===\n")
df_calls = drop_full_duplicates(df_calls, "df_calls")

print("\n=== 2. Преобразование формата даты и времени ===\n")
df_calls = convert_datetime_columns(
    df_calls,
    date_columns=["call_start_time"],
    df_name="df_calls"
)

print("\n=== 3. Очистка имён владельцев звонков ===\n")
df_calls = remove_boolean_artifacts(df_calls, "call_owner_name")

print("\n=== 4. Восстановление пропущенных contactid ===\n")
df_calls = restore_contactid_from_owner(df_calls, df_contacts)

print("\n=== 5. Очистка и нормализация категориальных столбцов ===\n")
cat_cols = ["call_type", "call_status", "outgoing_call_status"]
df_calls = normalize_categorical_columns(df_calls, cat_cols)

print("\n=== 6. Преобразование бинарного признака 'scheduled_in_crm' ===\n")
df_calls = process_scheduled_flag(df_calls, col="scheduled_in_crm")

print("\n=== 7. Обработка длительности звонков ===\n")
df_calls = fill_missing_numeric(df_calls, col="call_duration_in_seconds_", strategy="zero")
df_calls["call_duration_in_seconds_"] = df_calls["call_duration_in_seconds_"].astype("int32")
print("Столбец 'call_duration_in_seconds_' преобразован в int32")

print("\n=== 8. Удаление неинформативных столбцов ===\n")
df_calls = drop_empty_columns(df_calls, ["dialled_number", "tag"])

print("\n=== 9. Финальная проверка и выгрузка очищенного датасета ===\n")
print(f"[df_calls] Размер датафрейма: {df_calls.shape}\n")
print(df_calls.dtypes)

print("\n=== 10. Выгрузка очищенного датасета в .csv ===\n")
export_dataframe_to_csv(df_calls, "df_calls")


=== 1. Удаление полных дублей ===

[df_calls] Найдено полных дублей: 0
[df_calls] Полных дублей не обнаружено. Размер датафрейма: (95874, 11)

=== 2. Преобразование формата даты и времени ===

[df_calls] Столбец 'call_start_time' преобразован в datetime64[ns]. Некорректных значений: 0

=== 3. Очистка имён владельцев звонков ===

Артефактов False в 'call_owner_name' не обнаружено.

=== 4. Восстановление пропущенных contactid ===

Восстановление пропущенных contactid по contact_owner_name...
Восстановлено contactid: 3438 | Осталось пропусков: 495

=== 5. Очистка и нормализация категориальных столбцов ===

Столбец 'call_type' нормализован.


/tmp/ipython-input-2139470510.py:167: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(


Столбец 'call_status' нормализован.
Столбец 'outgoing_call_status' нормализован.

=== 6. Преобразование бинарного признака 'scheduled_in_crm' ===

Столбец 'scheduled_in_crm' преобразован в логический формат (0/1).
scheduled_in_crm
0    95732
1      142
Name: count, dtype: int64

=== 7. Обработка длительности звонков ===

'call_duration_in_seconds_': заполнено 83 пропусков (стратегия='zero')
Столбец 'call_duration_in_seconds_' преобразован в int32

=== 8. Удаление неинформативных столбцов ===

Удалены пустые/технические столбцы (если были): ['dialled_number', 'tag']

=== 9. Финальная проверка и выгрузка очищенного датасета ===

[df_calls] Размер датафрейма: (95874, 9)

id                           string[python]
call_start_time              datetime64[ns]
call_owner_name              string[python]
contactid                    string[python]
call_type                    string[python]
call_duration_in_seconds_             int32
call_status                  string[python]
outgoing_call_s

'/content/drive/MyDrive/P. Project 07.11/csv/df_calls_clean_20251102_1847.csv'

## Предобработка df_deals (таблица Deals (Done).xlsx)

|Этап предобработки|Необходимые действия|Ожидаемый результат|
|--|--|--|
|1. Проверка и удаление полных дублей|Проверить датафрейм на наличие полностью идентичных строк с помощью df_deals.drop_duplicates(). Удалить найденные дубликаты|Удалены повторяющиеся записи, каждая строка представляет уникальную сделку|
|2. Преобразование формата дат|Преобразовать столбцы created_time и closing_date к типу datetime64[ns] с указанием формата "%d.%m.%Y %H:%M" для created_time и "%d.%m.%Y" для closing_date|Корректные типы данных, обеспечивающие возможность анализа по времени|
|3. Проверка логики дат|Проверить, что closing_date не раньше created_time. При нарушениях зафиксировать или удалить строки|Данные согласованы по времени, отсутствуют нелогичные даты|
|4. Очистка имён владельцев сделок|Удалить лишние пробелы, заменить двойные пробелы одним, привести имена к единому регистру методом .str.title()|Единый формат имён менеджеров, готовый для сопоставления с другими таблицами|
|5. Проверка орфографических вариантов имён|Проверить уникальные значения deal_owner_name и при необходимости создать словарь нормализации для исправления опечаток и сокращений|Исправлены возможные опечатки и вариации написания, уменьшено количество уникальных имён|
|6. Преобразование числовых столбцов|Преобразовать столбцы initial_amount_paid и offer_total_amount к типу float, а course_duration и months_of_study — к целому типу Int64|Корректные числовые типы, готовые к аналитическим вычислениям|
|7. Очистка категориальных полей|Удалить пробелы по краям и внутри строк, привести текстовые значения к единому регистру для полей stage, quality, payment_type, product, education_type, source, campaign, content, term, city, level_of_deutsch|Единообразие категориальных данных, снижение числа дубликатов по формату записи|
|8. Проверка поля stage на корректность|Проверить, что значения stage соответствуют этапам воронки продаж (Lead, Contacted, Demo, Payment Process, Active Student, Churned). Исправить ошибки или неточные значения|Корректная структура этапов сделки, единая логика для анализа конверсий|
|9. Проверка финансовых показателей|Проверить, что offer_total_amount больше либо равно initial_amount_paid. При нарушениях поменять значения в найденных строках местами|Финансовые данные согласованы, исключены некорректные сделки|
|10. Проверка поля quality|Проверить категорию качества сделки (например, Hot, Warm, Cold) на наличие опечаток и приведение к единому регистру|Корректное распределение сделок по качеству, без дублирующих вариантов записи|
|11. Проверка логики обучения|Проверить, что months_of_study не превышает course_duration. При нарушениях отметить строки для анализа|Логически согласованные данные по продолжительности обучения|
|12. Финальная проверка типов данных|Убедиться, что все поля имеют корректные типы: даты — datetime64[ns], суммы — float, категории — string|Данные готовы для объединения с другими таблицами и анализа|
|13. Выгрузка очищенного набора данных в CSV|Сохранить очищенный датафрейм в файл с именем df_deals_clean_YYYYMMDD_HHMM.csv в папку проекта csv/ с кодировкой utf-8-sig|Создан файл с очищенными данными, готовый к дальнейшему использованию и визуализации|

Для того, чтобы оптимизировать процеудуру предобработки перенесем на этап EDA:
* анализ стадий без closing_date (распределение по stage);
* проверку логики дат (сравнение closing_date и created_time);
* проверку орфографических вариантов имён;
* логическую проверку числовых полей;
* проверку логики обучения.

### Очистка имён владельцев сделок

In [359]:
def clean_owner_names(df, col_name):
    """
    Очищает и стандартизирует имена владельцев (менеджеров) в CRM-таблице.
    Приводит текст к Title Case, удаляет лишние пробелы, нормализует регистр.
    """
    if col_name not in df.columns:
        print(f"Столбец '{col_name}' отсутствует — шаг пропущен.")
        return df

    df[col_name] = (
        df[col_name]
        .astype("string")
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
        .str.title()
    )

    unique_count = df[col_name].nunique(dropna=True)
    print(f"Очистка '{col_name}' выполнена. Уникальных имён: {unique_count}")
    return df

### Преобразование числовых полей

In [360]:
def convert_numeric_fields(df, float_cols, int_cols):
    """
    Преобразует числовые поля CRM к корректным типам данных.
    Для денежных значений:
      - удаляет разделители тысяч и лишние символы,
      - заменяет запятую на точку,
      - приводит к float.
    Для целых чисел:
      - конвертирует в pandas Int64 (nullable).
    """
    import numpy as np
    import pandas as pd

    # Денежные / вещественные столбцы
    for col in float_cols:
        if col in df.columns:
            df[col] = (
                df[col]
                .astype(str)
                .str.strip()
                .str.replace(r"(?<=\d)\.(?=\d{3}(\D|$))", "", regex=True)
                .str.replace(",", ".", regex=False)
                .str.replace(r"[^\d.]", "", regex=True)
                .replace(["", "nan", "NaN", "<NA>"], np.nan)
            )
            df[col] = pd.to_numeric(df[col], errors="coerce")
            print(f"'{col}' преобразован в float.")
        else:
            print(f"'{col}' отсутствует, пропущен.")

    # Целочисленные столбцы
    for col in int_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce").astype("Int64")
            print(f"'{col}' преобразован в Int64 (nullable).")
        else:
            print(f"'{col}' отсутствует, пропущен.")

    return df

### Коррекция перепутанных и отрицательных оплат

In [361]:
def correct_payment_values(df):
    """
    Исправляет ошибки в полях оплаты:
    - переставляет местами initial_amount_paid и offer_total_amount, если они перепутаны;
    - заменяет отрицательные значения initial_amount_paid на NA.
    """
    if not {"initial_amount_paid", "offer_total_amount"}.issubset(df.columns):
        print("Отсутствуют необходимые поля для проверки оплат.")
        return df

    # Перестановка перепутанных значений
    mask_swap = (
        (df["initial_amount_paid"] > df["offer_total_amount"]) &
        (df["offer_total_amount"].notna()) &
        (df["initial_amount_paid"].notna())
    )
    swap_count = mask_swap.sum()
    if swap_count > 0:
        df.loc[mask_swap, ["initial_amount_paid", "offer_total_amount"]] = (
            df.loc[mask_swap, ["offer_total_amount", "initial_amount_paid"]].values
        )
        print(f"Перестановка значений выполнена в {swap_count} строках.")
    else:
        print("Перепутанных значений не обнаружено.")

    # Замена отрицательных значений на пропуски
    mask_negative = df["initial_amount_paid"] < 0
    neg_count = mask_negative.sum()
    if neg_count > 0:
        df.loc[mask_negative, "initial_amount_paid"] = pd.NA
        print(f"Отрицательные значения 'initial_amount_paid' заменены на NA ({neg_count} строк).")
    else:
        print("Отрицательных значений не обнаружено.")

    return df

### Нормализация стадий сделок

In [362]:
df_deals["stage"].unique()

array(['New Lead', 'Lost', 'Need a consultation', 'Need To Call',
       'Call Delayed', 'Qualificated', 'Registered on Webinar',
       'Waiting For Payment', 'Need to Call - Sales', 'Test Sent',
       'Payment Done', 'Registered on Offline Day', 'Free Education'],
      dtype=object)

In [363]:
def map_stage_to_funnel(df):
    """
    Создаёт поле stage_normalized на основе CRM-стадий сделки,
    приводя все варианты к стандартным этапам воронки онлайн-школы:
    Lead → Contacted → Demo → Payment Process → Active Student → Churned
    """
    if "stage" not in df.columns:
        print("Поле 'stage' отсутствует — шаг пропущен.")
        return df

    stage_mapping = {
        "new lead": "Lead",
        "call delayed": "Lead",
        "registered on webinar": "Lead",
        "registered on offline day": "Lead",

        "contact": "Contacted",
        "need to call": "Contacted",
        "need to call sales": "Contacted",
        "qualificated": "Contacted",

        "demo call": "Demo",
        "trial lesson": "Demo",
        "need a consultation": "Demo",
        "test sent": "Demo",

        "payment": "Payment Process",
        "payment in progress": "Payment Process",
        "waiting for payment": "Payment Process",

        "paid": "Active Student",
        "student": "Active Student",
        "active": "Active Student",
        "payment done": "Active Student",
        "free education": "Active Student",

        "closed lost": "Churned",
        "lost": "Churned",
        "inactive": "Churned",
    }

    df["stage_normalized"] = (
        df["stage"]
        .astype("string")
        .str.strip()
        .str.lower()
        .str.replace("\u00a0", " ", regex=False)
        .str.replace(r"[-_–—]+", " ", regex=True)
        .str.replace(r"\s+", " ", regex=True)
        .map(stage_mapping)
        .fillna("Undefined")
    )

    print("Нормализация поля 'stage' завершена. Пропуски заполнены 'Undefined'.")
    return df

### Нормализация качества лидов

In [364]:
df_deals["quality"].unique()

array([nan, 'D - Non Target', 'E - Non Qualified', 'B - Medium',
       'C - Low', 'A - High', 'F'], dtype=object)

In [365]:
def map_quality_to_rating(df):
    """
    Создаёт поле quality_normalized на основе рейтинга A–E/F.
    Приводит их к универсальным уровням: Hot / Warm / Cold / Undefined.
    """
    if "quality" not in df.columns:
        print("Поле 'quality' отсутствует — шаг пропущен.")
        return df

    quality_mapping = {
        "a high": "Hot",
        "b medium": "Warm",
        "c low": "Cold",
        "d non target": "Cold",
        "e non qualified": "Undefined",
        "f": "Undefined",
    }

    df["quality_normalized"] = (
        df["quality"]
        .astype("string")
        .str.strip()
        .str.lower()
        .str.replace("\u00a0", " ", regex=False)     # неразрывные пробелы
        .str.replace(r"[-_]+", " ", regex=True)     # все тире и подчёркивания → пробел
        .str.replace(r"\s+", " ", regex=True)       # двойные пробелы → один
        .map(quality_mapping)
        .fillna("Undefined")
        .str.title()
    )

    print("Нормализация поля 'quality' завершена. Пропуски заполнены 'Undefined'.")
    return df


### Предобработка df_deals и выгрузка в csv

In [366]:
print("=== 1. Удаление полных дублей ===\n")
df_deals = drop_full_duplicates(df_deals, "df_deals")

print("\n=== 2. Преобразование формата дат ===\n")
df_deals = convert_datetime_columns(
    df_deals,
    date_columns=["created_time", "closing_date"],
    df_name="df_deals"
)

print("\n=== 3. Очистка и стандартизация имён владельцев сделок ===\n")
df_deals = remove_boolean_artifacts(df_deals, "deal_owner_name")
df_deals = clean_owner_names(df_deals, "deal_owner_name")

print("\n=== 4. Преобразование числовых полей ===\n")
float_cols = ["initial_amount_paid", "offer_total_amount"]
int_cols = ["course_duration", "months_of_study"]
df_deals = convert_numeric_fields(df_deals, float_cols, int_cols)

print("\n=== 5. Очистка категориальных полей ===\n")
cat_cols = [
    "stage", "quality", "payment_type", "product",
    "education_type", "source", "campaign",
    "content", "term", "city", "level_of_deutsch", "page", "lost_reason"
]
df_deals = normalize_categorical_columns(df_deals, cat_cols)

print("\n=== 6. Коррекция перепутанных и отрицательных значений оплаты ===\n")
df_deals = correct_payment_values(df_deals)

print("\n=== 7. Нормализация стадий сделок (stage в stage_normalized) ===\n")
df_deals = map_stage_to_funnel(df_deals)

print("\n=== 8. Нормализация рейтинга качества (quality в quality_normalized) ===\n")
df_deals = map_quality_to_rating(df_deals)

print("\n=== 9. Приведение типов строковых полей ===\n")
object_cols = df_deals.select_dtypes(include="object").columns.tolist()
if object_cols:
    df_deals[object_cols] = df_deals[object_cols].astype("string")
    print(f"Преобразовано столбцов типа 'object' в string: {len(object_cols)}")
else:
    print("Все строковые поля уже имеют тип string.")

print("\n=== 10. Финальная проверка и выгрузка очищенного датасета ===\n")
print(f"[df_deals] Размер датафрейма: {df_deals.shape}\n")
print(df_deals.dtypes)

print("\n=== 11. Выгрузка очищенного набора данных ===\n")
export_dataframe_to_csv(df_deals, "df_deals")


=== 1. Удаление полных дублей ===

[df_deals] Найдено полных дублей: 0
[df_deals] Полных дублей не обнаружено. Размер датафрейма: (21593, 23)

=== 2. Преобразование формата дат ===

[df_deals] Столбец 'created_time' преобразован в datetime64[ns]. Некорректных значений: 0
[df_deals] Столбец 'closing_date' преобразован в datetime64[ns]. Некорректных значений: 6948

=== 3. Очистка и стандартизация имён владельцев сделок ===

Артефактов False в 'deal_owner_name' не обнаружено.
Очистка 'deal_owner_name' выполнена. Уникальных имён: 27

=== 4. Преобразование числовых полей ===



/tmp/ipython-input-2139470510.py:167: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(
/tmp/ipython-input-2139470510.py:167: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(


'initial_amount_paid' преобразован в float.
'offer_total_amount' преобразован в float.
'course_duration' преобразован в Int64 (nullable).
'months_of_study' преобразован в Int64 (nullable).

=== 5. Очистка категориальных полей ===

Столбец 'stage' нормализован.
Столбец 'quality' нормализован.
Столбец 'payment_type' нормализован.
Столбец 'product' нормализован.
Столбец 'education_type' нормализован.
Столбец 'source' нормализован.
Столбец 'campaign' нормализован.
Столбец 'content' нормализован.
Столбец 'term' нормализован.
Столбец 'city' нормализован.
Столбец 'level_of_deutsch' нормализован.
Столбец 'page' нормализован.
Столбец 'lost_reason' нормализован.

=== 6. Коррекция перепутанных и отрицательных значений оплаты ===

Перестановка значений выполнена в 58 строках.
Отрицательных значений не обнаружено.

=== 7. Нормализация стадий сделок (stage в stage_normalized) ===

Нормализация поля 'stage' завершена. Пропуски заполнены 'Undefined'.

=== 8. Нормализация рейтинга качества (quality в q

'/content/drive/MyDrive/P. Project 07.11/csv/df_deals_clean_20251102_1847.csv'

**Проверка полученных нормализованных категориальных переменных**

In [367]:
df_deals["stage_normalized"].unique()

<StringArray>
['Lead', 'Churned', 'Demo', 'Contacted', 'Payment Process', 'Active Student']
Length: 6, dtype: string

In [368]:
df_deals["quality_normalized"].unique()

<StringArray>
['Undefined', 'Cold', 'Warm', 'Hot']
Length: 4, dtype: string

## Предобработка и обогащение метриками df_spend (таблица Spend (Done).xlsx')

|Этап предобработки|Необходимые действия|Ожидаемый результат|
|--|--|--|
|1. Проверка и удаление полных дублей|Проверить наличие полностью идентичных строк с помощью метода duplicated(). Удалить найденные дубликаты и сбросить индексы|Удалены повторяющиеся записи, каждая строка представляет уникальное наблюдение|
|2. Преобразование формата даты|Преобразовать столбец date в формат datetime64[ns], указав формат '%Y-%m-%d' или определить автоматически с errors='coerce'|Корректное хранение даты, возможность группировки и анализа по периодам|
|3. Очистка текстовых полей|Удалить лишние пробелы и дублирующие символы в текстовых столбцах source, campaign, adgroup, ad. Привести значения к единому регистру (например, Title Case)|Единообразие текстовых данных, исключение ошибок при объединении и фильтрации|
|4. Преобразование числовых полей|Преобразовать столбцы impressions, spend и clicks в числовой формат (int или float). Заменить нечисловые символы и запятые. Проверить диапазоны на наличие отрицательных значений|Корректные типы данных для расчётов и агрегирования|
|5. Проверка логики числовых полей|Убедиться, что impressions, clicks и spend неотрицательны. Проверить, что clicks ≤ impressions. При нарушении заменить значения на NaN или удалить строки|Данные логически согласованы, исключены ошибки загрузки и импорта|
|6. Проверка консистентности кампаний|Проверить уникальные сочетания source, campaign и adgroup. Убедиться, что одно объявление (ad) не относится к нескольким источникам. Исправить несоответствия или пометить для анализа|Согласованность связей между источниками, кампаниями и объявлениями|
|7. Проверка временных дубликатов|Проверить, нет ли повторяющихся записей по комбинации date + source + campaign + adgroup + ad. При обнаружении агрегировать значения impressions, clicks и spend по сумме|Устранены повторяющиеся строки за одну дату, корректная ежедневная агрегированная статистика|
|8. Добавление вычисляемых метрик|Рассчитать CTR (clicks / impressions * 100) и CPC (spend / clicks). Добавить их как отдельные столбцы, при clicks=0 проставить NaN|Получены дополнительные аналитические показатели для оценки эффективности рекламы|
|9. Финальная проверка типов данных|Проверить типы всех столбцов: даты — datetime64[ns], числовые — float или int, категории — string. При необходимости привести типы к корректным|Данные полностью готовы к аналитике и выгрузке|
|10. Выгрузка очищенного набора данных в CSV|Сохранить очищенный датафрейм в файл df_spend_clean_YYYYMMDD_HHMM.csv в папку проекта csv/ с кодировкой utf-8-sig, используя кастомную функцию export_to_csv()|Создан файл с очищенными и подготовленными данными, готовыми для визуализации и анализа|

Для того, чтобы оптимизировать процеудуру предобработки перенесем на этап EDA:
* проверку диапазонов числовых значений;
* проверку логики числовых полей (отрицательные значения, clicks > impressions, статистика);
* проверку консистентности кампаний (ad в нескольких source/campaign);
* агрегацию и устранение временных дубликатов.

Для упрощения создания дашборда и проведения product-анализа обогатим исходный датасет следующими продуктовыми метриками:
* CTR (Click-Through Rate, %)
* CPC (Cost Per Click)
* CPM (Cost Per Mille)
* Efficiency Index — эффективность кампании относительно затрат.


### Расчёт CTR и CPC

In [369]:
def calculate_ctr_cpc(df):
    """
    Добавляет вычисляемые столбцы CTR (Click-Through Rate, %)
    и CPC (Cost Per Click) в датафрейм df_spend.
    - CTR = (clicks / impressions) * 100
    - CPC = spend / clicks
    """
    df = df.copy()

    df["ctr"] = np.where(
        (df["impressions"] > 0) & (df["clicks"].notna()),
        (df["clicks"] / df["impressions"]) * 100,
        np.nan
    )

    df["cpc"] = np.where(
        (df["clicks"] > 0) & (df["spend"].notna()),
        df["spend"] / df["clicks"],
        np.nan
    )

    # Очистка нелогичных значений
    df.loc[df["ctr"] > 100, "ctr"] = np.nan
    df.loc[df["cpc"] < 0, "cpc"] = np.nan

    print("Добавлены вычисляемые метрики CTR и CPC.")
    return df

### Расчёт CPM и Efficiency Index

In [370]:
def calculate_cpm_efficiency(df):
    """
    Добавляет метрики CPM (Cost Per Mille) и Efficiency Index.
    - CPM = (spend / impressions) * 1000
    - Efficiency Index = CTR / CPC (чем выше, тем эффективнее реклама)
    """
    df = df.copy()

    df["cpm"] = np.where(
        (df["impressions"] > 0) & (df["spend"].notna()),
        (df["spend"] / df["impressions"]) * 1000,
        np.nan
    )

    df["efficiency_index"] = np.where(
        (df["cpc"] > 0) & (df["ctr"].notna()),
        df["ctr"] / df["cpc"],
        np.nan
    )

    print("Добавлены метрики CPM и Efficiency Index.")
    return df

Предобработка df_spend и выгрузка в csv

In [371]:
print("=== 1. Удаление полных дублей ===\n")
df_spend = drop_full_duplicates(df_spend, "df_spend")

print("\n=== 2. Преобразование формата даты ===\n")
df_spend = convert_datetime_columns(
    df_spend,
    date_columns=["date"],
    df_name="df_spend"
)

print("\n=== 3. Очистка текстовых полей ===\n")
text_cols = ["source", "campaign", "adgroup", "ad"]
df_spend = normalize_categorical_columns(df_spend, text_cols)

print("\n=== 4. Преобразование числовых полей ===\n")

# Определяем числовые столбцы
int_cols = ["impressions", "clicks"]
float_cols = ["spend"]

# Преобразование числовых значений
for col in int_cols:
    if col in df_spend.columns:
        df_spend[col] = (
            df_spend[col]
            .astype(str)
            .str.replace(r"[^\d]", "", regex=True)
            .replace("", pd.NA)
        )
        df_spend[col] = pd.to_numeric(df_spend[col], errors="coerce").astype("Int64")
        print(f"'{col}' преобразован в Int64 (nullable).")

for col in float_cols:
    if col in df_spend.columns:
        df_spend[col] = (
            df_spend[col]
            .astype(str)
            .str.replace(",", ".", regex=False)
            .str.replace(r"(?<=\d)\.(?=\d{3}(\D|$))", "", regex=True)
            .str.replace(r"[^\d.]", "", regex=True)
            .replace("", pd.NA)
        )
        df_spend[col] = pd.to_numeric(df_spend[col], errors="coerce")
        print(f"'{col}' преобразован в float.")

print("\n=== 5. Очистка отрицательных и нелогичных значений ===\n")

# Удаляем отрицательные значения
num_cols = ["impressions", "clicks", "spend"]
for col in num_cols:
    if col in df_spend.columns:
        neg_count = (df_spend[col] < 0).sum()
        if neg_count > 0:
            df_spend.loc[df_spend[col] < 0, col] = pd.NA
            print(f"Заменено отрицательных значений в '{col}': {neg_count}")
        else:
            print(f"Отрицательных значений в '{col}' не обнаружено.")

# Корректируем логические несоответствия clicks > impressions
mask_invalid = (df_spend["clicks"].notna()) & (df_spend["impressions"].notna()) & (df_spend["clicks"] > df_spend["impressions"])
affected_rows = mask_invalid.sum()
if affected_rows > 0:
    df_spend.loc[mask_invalid, "clicks"] = df_spend.loc[mask_invalid, "impressions"]
    print(f"Исправлено строк, где clicks > impressions: {affected_rows}")
else:
    print("Логических несоответствий clicks > impressions не обнаружено.")

print("\n=== 6. Приведение типов строковых полей ===\n")
object_cols = df_spend.select_dtypes(include="object").columns.tolist()
if object_cols:
    df_spend[object_cols] = df_spend[object_cols].astype("string")
    print(f"Преобразовано столбцов типа 'object' в string: {len(object_cols)}")
else:
    print("Все строковые поля уже имеют тип string.")

print("\n=== 7. Обогащение датафрейма маркетинговыми метриками ===\n")

# --- CTR (Click-Through Rate) ---
df_spend["ctr"] = np.where(
    (df_spend["impressions"] > 0) & (df_spend["clicks"].notna()),
    (df_spend["clicks"] / df_spend["impressions"]) * 100,
    np.nan
)

# --- CPC (Cost Per Click) ---
df_spend["cpc"] = np.where(
    (df_spend["clicks"] > 0) & (df_spend["spend"].notna()),
    df_spend["spend"] / df_spend["clicks"],
    np.nan
)

# --- CPM (Cost Per Mille) ---
df_spend["cpm"] = np.where(
    (df_spend["impressions"] > 0) & (df_spend["spend"].notna()),
    (df_spend["spend"] / df_spend["impressions"]) * 1000,
    np.nan
)

# --- Efficiency Index (CTR / CPC) ---
df_spend["efficiency_index"] = np.where(
    (df_spend["cpc"] > 0) & (df_spend["ctr"].notna()),
    df_spend["ctr"] / df_spend["cpc"],
    np.nan
)

# Очистка нелогичных значений
df_spend.loc[df_spend["ctr"] > 100, "ctr"] = np.nan
df_spend.loc[df_spend["cpc"] < 0, "cpc"] = np.nan
df_spend.loc[df_spend["cpm"] < 0, "cpm"] = np.nan
df_spend.loc[df_spend["efficiency_index"] < 0, "efficiency_index"] = np.nan

print("Добавлены метрики: CTR, CPC, CPM, Efficiency Index.")

print("\n=== 8. Финальная проверка и выгрузка очищенного датасета ===\n")
print(f"[df_spend] Размер датафрейма: {df_spend.shape}\n")
print(df_spend.dtypes)

print("\n=== 9. Выгрузка очищенного набора данных ===\n")
export_dataframe_to_csv(df_spend, "df_spend")

=== 1. Удаление полных дублей ===

[df_spend] Найдено полных дублей: 917
[df_spend] Полные дубли удалены. Размер после очистки: (19862, 8)

=== 2. Преобразование формата даты ===

[df_spend] Столбец 'date' преобразован в datetime64[ns]. Некорректных значений: 0

=== 3. Очистка текстовых полей ===

Столбец 'source' нормализован.
Столбец 'campaign' нормализован.
Столбец 'adgroup' нормализован.
Столбец 'ad' нормализован.

=== 4. Преобразование числовых полей ===

'impressions' преобразован в Int64 (nullable).
'clicks' преобразован в Int64 (nullable).


/tmp/ipython-input-2139470510.py:167: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[col] = pd.to_datetime(


'spend' преобразован в float.

=== 5. Очистка отрицательных и нелогичных значений ===

Отрицательных значений в 'impressions' не обнаружено.
Отрицательных значений в 'clicks' не обнаружено.
Отрицательных значений в 'spend' не обнаружено.
Исправлено строк, где clicks > impressions: 1363

=== 6. Приведение типов строковых полей ===

Все строковые поля уже имеют тип string.

=== 7. Обогащение датафрейма маркетинговыми метриками ===

Добавлены метрики: CTR, CPC, CPM, Efficiency Index.

=== 8. Финальная проверка и выгрузка очищенного датасета ===

[df_spend] Размер датафрейма: (19862, 12)

date                datetime64[ns]
source              string[python]
campaign            string[python]
impressions                  Int64
spend                      float64
clicks                       Int64
adgroup             string[python]
ad                  string[python]
ctr                        float64
cpc                        float64
cpm                        float64
efficiency_index       

'/content/drive/MyDrive/P. Project 07.11/csv/df_spend_clean_20251102_1847.csv'

## Перенести в EDA

### Сопоставление владельцев контактов, звонков и сделок

In [372]:
print("=== Таблица сопоставления владельцев контактов, звонков и сделок ===\n")

# Получаем уникальные имена из всех трёх таблиц
contacts_owners = pd.Series(df_contacts["contact_owner_name"].unique(), name="contact_owner_name")
calls_owners = pd.Series(df_calls["call_owner_name"].unique(), name="call_owner_name")
deals_owners = pd.Series(df_deals["deal_owner_name"].unique(), name="deal_owner_name")

# Приведём к DataFrame и объединим для анализа
mapping_df = (
    pd.DataFrame(contacts_owners)
    .merge(pd.DataFrame(calls_owners), left_on="contact_owner_name", right_on="call_owner_name", how="outer")
    .merge(pd.DataFrame(deals_owners), left_on="contact_owner_name", right_on="deal_owner_name", how="outer", indicator=True)
)

# Сортировка для удобства анализа
mapping_df = mapping_df.sort_values(by="contact_owner_name", na_position="last").reset_index(drop=True)

# Итоги по количеству уникальных имён в каждом датасете
print(f"Всего уникальных имён в Contacts: {df_contacts['contact_owner_name'].nunique()}")
print(f"Всего уникальных имён в Calls: {df_calls['call_owner_name'].nunique()}")
print(f"Всего уникальных имён в Deals: {df_deals['deal_owner_name'].nunique()}")

# Поиск несовпадающих записей между тремя таблицами
non_matched = mapping_df[
    mapping_df["contact_owner_name"].isna()
    | mapping_df["call_owner_name"].isna()
    | mapping_df["deal_owner_name"].isna()
]

print(f"\nВсего несопоставленных записей: {len(non_matched)}")
print("\nПримеры несопоставленных записей:")
print(non_matched.head(15))

# Показываем итоговую таблицу сопоставления
print("\nТаблица сопоставления владельцев контактов, звонков и сделок (первые 40 строк):\n")
mapping_df.head(40)


=== Таблица сопоставления владельцев контактов, звонков и сделок ===

Всего уникальных имён в Contacts: 27
Всего уникальных имён в Calls: 33
Всего уникальных имён в Deals: 27

Всего несопоставленных записей: 10

Примеры несопоставленных записей:
   contact_owner_name call_owner_name deal_owner_name      _merge
6         Derek James     Derek James            <NA>   left_only
21         Tina Zhang      Tina Zhang            <NA>   left_only
27               <NA>            <NA>        John Doe  right_only
28               <NA>            <NA>     Xander Dean  right_only
29               <NA>    Ethan Harris            <NA>        both
30               <NA>   Fiona Jackson            <NA>        both
31               <NA>      Hannah Lee            <NA>        both
32               <NA>        John Doe            <NA>        both
33               <NA>     Laura Quinn            <NA>        both
34               <NA>     Xander Dean            <NA>        both

Таблица сопоставления владе

,contact_owner_name,call_owner_name,deal_owner_name,_merge
0,Alice Johnson,Alice Johnson,Alice Johnson,both
1,Amy Green,Amy Green,Amy Green,both
2,Ben Hall,Ben Hall,Ben Hall,both
3,Bob Brown,Bob Brown,Bob Brown,both
4,Cara Iverson,Cara Iverson,Cara Iverson,both
5,Charlie Davis,Charlie Davis,Charlie Davis,both
6,Derek James,Derek James,<NA>,left_only
7,Diana Evans,Diana Evans,Diana Evans,both
8,Eva Kent,Eva Kent,Eva Kent,both
9,George King,George King,George King,both
